In [1]:
import svhnFileReader as sv
import numpy as np
import TrainSVHN as ts
import importlib

## loading files

In [2]:
# Dataset download and confirming data and functions
# ==================================================

# reload dependencies 
importlib.reload(ts)
importlib.reload(sv)

files = ["train", "test", "extra"]

# download svhn files
for file in files:
    sv.maybeDownload(file)

# choose some random indieces
data_samples = np.random.permutation(400)[:20]
print("The random file indexes", data_samples) # Looking good?

# get file names and labels associated with random indieces
train_files, train_labels = sv.getLabels(
    'train/digitStruct.mat', data_samples)

# check the file names and labels
print("Train files are", train_files)
print("Label of train files are", train_labels)

# get the data in file names
data = sv.getImage(train_files, 'train/', shape=(80, 40))
print("Shape of train data is",data.shape) # what is the size? does it match?
print("Min and Max of data are", np.min(data), np.max(data))
print("Train labels are before parsing:\n", train_labels[:15])
print("Train labels are after parsing:\n", sv.parseLabels(train_labels[:15],3))

# show some of the data
# sv.showMultipleArraysHorizontally(data[:15], train_labels[:15], 3)


train already exists
test already exists
extra already exists
The random file indexes [370  94  80  72  84  81 260   1 287  51 167 263 180  48 274  42  67  68
 330 123]
Train files are ['371.png', '95.png', '81.png', '73.png', '85.png', '82.png', '261.png', '2.png', '288.png', '52.png', '168.png', '264.png', '181.png', '49.png', '275.png', '43.png', '68.png', '69.png', '331.png', '124.png']
Label of train files are [[8.0], [1.0, 10.0, 4.0, 4.0], [7.0, 4.0], [3.0], [1.0, 7.0], [4.0, 5.0], [1.0, 4.0], [2.0, 3.0], [2.0, 9.0, 2.0], [3.0, 5.0], [2.0, 8.0, 2.0], [2.0, 1.0, 9.0, 10.0], [3.0, 7.0], [8.0, 1.0, 7.0], [2.0, 5.0], [2.0, 9.0], [5.0, 4.0, 2.0], [4.0, 4.0], [2.0, 3.0], [5.0]]
Shape of train data is (20, 40, 80, 3)
Min and Max of data are 0.0 255.0
Train labels are before parsing:
 [[8.0], [1.0, 10.0, 4.0, 4.0], [7.0, 4.0], [3.0], [1.0, 7.0], [4.0, 5.0], [1.0, 4.0], [2.0, 3.0], [2.0, 9.0, 2.0], [3.0, 5.0], [2.0, 8.0, 2.0], [2.0, 1.0, 9.0, 10.0], [3.0, 7.0], [8.0, 1.0, 7.0], [2.0, 5.0]

In [4]:
# Preprocessing
# =============

# dependencies
importlib.reload(ts)
importlib.reload(sv)

# configurations
big_batch_size = 2000
image_shape = (80,40)
max_digits_in_label = 4

def preprocess(dataset):
    # read lots of files
    pickle_file = dataset+"_preprocessed"
    struct_file = dataset+"/digitStruct.mat"
    number_of_files = sv.getNumberOfFiles(struct_file)
#     number_of_files = big_batch_size # just for debug
    data_samples = np.random.permutation(number_of_files)
    file_handle = open(pickle_file,"wb")

    # iterate over data in big batches
    for batch_start in range(0,number_of_files, big_batch_size):

        # read the .mat file and parse attributes of data files
        batch_indexes = data_samples[batch_start:batch_start+big_batch_size]

        file_names,train_labels = sv.getLabels(struct_file,batch_indexes)
        train_values = sv.getImage(file_names, dataset,shape=image_shape)


        # form and normalize
        pixel_depth = 255
        train_values = sv.scaleData(train_values,pixel_depth)
        train_labels = sv.parseLabels(train_labels,max_digits_in_label)

        # save in file
        np.save(file_handle, train_values)
        np.save(file_handle, train_labels)

        # process status
        completion_percentil = 100*(batch_start+big_batch_size)/number_of_files
        print("Compeleted %%%d"%completion_percentil)

    # always close the file
    file_handle.close()
    
# perform preprocessing
# preprocess('train')
# preprocess('test')

In [5]:
# Data extractor
# ==============

def dataGenerator(batch_size,file_name):
    file_handle = open(file_name, "rb")
    while True:

        # get data array
        try:
            data = np.load(file_handle)
        # if reached end of file
        except OSError:
#             print("in dataGenerator() pointer is at",file_handle.tell(),"... going back.")
            # go to the beginning
            file_handle.seek(0)
            # and try loading again
            data = np.load(file_handle)

        # get label array
        labels = np.load(file_handle)
        
        # randomize
        data,labels = sv.shuffleArrays([data,labels])
        
        # get batches        
        number_of_datapoints = labels.shape[0]
        full_batches = number_of_datapoints//batch_size # few datapoints are going to waste here
        start_point = 0
        for batch_start in range(0,full_batches,batch_size):
            batch_data = data[batch_start:batch_start+batch_size]
            batch_labels = labels[batch_start:batch_start+batch_size]
            
            # yield both
            yield batch_data, batch_labels

In [6]:
# validate dataGenerator and disk data
# ====================================

importlib.reload(sv)

train_file = "train_preprocessed"
gen = dataGenerator(3,train_file)
sample_data,sample_labels = next(gen)

print(sv.multipleOneHots(sample_labels,[max_digits_in_label+1]+[11]*max_digits_in_label))
# sv.showMultipleArraysHorizontally(sample_data+.5,sample_labels,3)
del gen

[[ 0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [7]:
def oneHotsToLabels(onehots,class_sizes):
    offset=0
    labels = np.zeros((len(onehots),len(class_sizes)),int)
    for i in range(len(class_sizes)):
        labels[:,i]=np.argmax(onehots[:,offset:offset+class_sizes[i]],1)
        offset+=class_sizes[i]
    return labels

one_hots = sv.multipleOneHots(sample_labels,[max_digits_in_label+1]+[11]*max_digits_in_label)
print(oneHotsToLabels(one_hots,[max_digits_in_label+1]+[11]*max_digits_in_label))

# sv.showMultipleArraysHorizontally(sample_data+.5,sample_labels,3)

[[2 2 1 0 0]
 [2 3 4 0 0]
 [2 3 4 0 0]]


In [ ]:
# Make model
# ==========

# dependencies
importlib.reload(ts)

# configurations
number_of_steps = 100
batch_size = 8
depth_conv = [48, 64]
strides = [2,1]
depth_fully_connected = [10, 10]
initial_learning_rate = 1e-3
decay = .9
report_steps = number_of_steps // 10
size_of_classes = [max_digits_in_label+1]+[11]*max_digits_in_label


# model
network = ts.SVHNTrainer()

# apply config
network.depth_conv = depth_conv
network.max_pool_strides = strides
network.depth_fully_connected = depth_fully_connected
network.num_channels = 3
network.report_step = report_steps
network.initial_learning_rate = initial_learning_rate
network.decay = decay
network.image_height = image_shape[1]
network.image_width = image_shape[0]
network.batch_size = batch_size
network.class_sizes = size_of_classes
network.output_neurons = sum(network.class_sizes)

network.makeGraph()

# define the generator
def dataMaker(batch_size):
    gen = dataGenerator(batch_size,train_file)
    while True:
        data, labels = next(gen)
        labels = sv.multipleOneHots(labels,size_of_classes)
        yield data, labels

    
# test generator
# --------------
# generator = dataMaker(3)
# sample_gen_data, sample_gen_label = next(generator)
# print(sample_gen_label)
# sv.showMultipleArraysHorizontally(sample_gen_data+.5)



{'num_channels': 3, 'report_step': 200, 'class_sizes': [5, 11, 11, 11, 11], 'max_pool_strides': [2, 1, 2, 1, 2, 1], 'image_width': 80, 'image_height': 40, 'depth_conv': [48, 64, 128, 160, 192, 192], 'graph': <tensorflow.python.framework.ops.Graph object at 0x7f8e1f9e2208>, 'batch_size': 8, 'depth_fully_connected': [1000, 1000], 'report_string': '----------', 'output_neurons': 49, 'decay': 0.9, 'initial_learning_rate': 0.001}
Weights of convolution layers have the following sizes:
(5, 5, 3, 48) maxpool stride: 2
(5, 5, 48, 64) maxpool stride: 1
(5, 5, 64, 128) maxpool stride: 2
(5, 5, 128, 160) maxpool stride: 1
(5, 5, 160, 192) maxpool stride: 2
(5, 5, 192, 192) maxpool stride: 1
Data has the following shapes between convolutional layers:
(8, 40, 80, 3)
(8, 20, 40, 48)
(8, 20, 40, 64)
(8, 10, 20, 128)
(8, 10, 20, 160)
(8, 5, 10, 192)
Data's shape before flat layers is: (8, 9600)
Data has following shapes between flat layers:
(8, 1000)
(8, 1000)
Final shape of data is: (8, 49)
Seperated

In [ ]:
# Train Model
# ===========

# dependencies
importlib.reload(ts)

# configurations
validation_steps = 10
test_steps = 100

# train model
del generator
generator = dataMaker(network.batch_size)
prediction_sample = network.train(number_of_steps,generator,validation_steps,test_steps)

In [15]:
network.saveReport()

Saved to: model_report_2016-09-21_15:22:23


In [40]:
# Check test results
# ==================


predicted_labels = oneHotsToLabels(prediction_sample[1],size_of_classes)
print(predicted_labels)
sv.showMultipleArraysHorizontally(prediction_sample[0]+.5,predicted_labels,3)


[[2 1 0 0 0]
 [2 1 0 0 0]
 [2 1 0 0 0]
 [2 1 0 0 0]
 [2 1 0 0 0]
 [2 1 0 0 0]
 [2 1 0 0 0]
 [2 1 0 0 0]]
